In [1]:
import intrinio
import pandas as pd
import numpy as np
import requests.exceptions
import sys

In [4]:
#I couldnt figure out how to use the api so i downloaded AAPL from corbins github, just change this part to include
#the access to the api, and if you guys find a different/better stock this should also work with that
temp = pd.read_csv('AAPL.csv')

In [28]:
listsOfHighs = dict()
for i in range(30,len(temp)-30):
    listsOfHighs[i] = []
    
for i , row in temp.iterrows():
    if i in range(30,len(temp)-30):
        for j in range(i-30,i):
            listsOfHighs[i].append(temp.iloc[j].adj_high)


In [41]:
#listsOfHighs is just a dict with lists of the last 30 days highs

In [38]:
#adds col for 30 day avg and std dev, and fills in values
temp['30_avg'] = np.NaN
temp['30_std'] = np.NaN
for i ,row in temp.iterrows():
    if i in range(30,len(temp)-30):
        temp.set_value(i,'30_avg',sum(listsOfHighs[i])/len(listsOfHighs[i]))
        temp.set_value(i,'30_std',np.std(np.array(listsOfHighs[i])))
    

temp

In [45]:
temp.iloc[[30]]
#so it works

,Unnamed: 0,adj_close,adj_high,adj_low,adj_open,adj_volume,close,ex_dividend,high,low,open,split_ratio,volume,30_avg,30_std
30,2016-12-28,116.259562,117.510777,115.701963,117.016305,20905892.0,116.76,0.0,118.0166,116.2,117.52,1.0,20905892.0,122.218613,5.43085
